In [1]:
import pandas as pd
import psycopg
from dotenv import load_dotenv
import os

# === 1. .env laden ===
load_dotenv()
dbconn = os.getenv("SUPABASE_DB_URL")

# === 2. CSV laden und vorbereiten ===
df = pd.read_csv("../data/EDM_B4S.csv", sep=";")
df.columns = [col.strip().replace(" ", "_") for col in df.columns]
df["time"] = pd.to_datetime(df["time"], errors="coerce")
df = df.dropna(subset=["time"])
df = df.head(10)  # Nur erste 200 Zeilen

# === 3. Verbindung zur Supabase-PostgreSQL ===
conn = psycopg.connect(dbconn)
cur = conn.cursor()

# === 4. Tabelle erstellen, falls nicht vorhanden ===
cur.execute('''
    CREATE TABLE IF NOT EXISTS b4s_data (
        time TIMESTAMP PRIMARY KEY,
        Nettolast_P_kW REAL,
        Nettolast_Q_kvar REAL
    );
''')

# === 5. Zeilen als Tupelliste vorbereiten ===
rows = [
    (
        row["time"],
        float(row["Nettolast_P_kW"]) if pd.notna(row["Nettolast_P_kW"]) else None,
        float(row["Nettolast_Q_kvar"]) if pd.notna(row["Nettolast_Q_kvar"]) else None
    )
    for _, row in df.iterrows()
]

# === 6. Schneller Bulk-Insert mit Konfliktbehandlung ===
cur.executemany('''
    INSERT INTO b4s_data (time, Nettolast_P_kW, Nettolast_Q_kvar)
    VALUES (%s, %s, %s)
    ON CONFLICT (time) DO NOTHING;
''', rows)

# === 7. Speichern und Verbindung schließen ===
conn.commit()
cur.close()
conn.close()

print("✅ Erfolgreich 200 Zeilen in Supabase hochgeladen.")


✅ Erfolgreich 200 Zeilen in Supabase hochgeladen.
